# WorkFlow

## Classes

## Load the data

## Test Modelling

## Modelling

**<hr>**

## Classes

In [69]:
NAME = ""

In [70]:
BATCH_SIZE = 32

In [71]:
import os
import cv2
import torch
import numpy as np

In [72]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [73]:
import torch

In [74]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

**<hr>**

## Load the data

In [75]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

36
2515


/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


going through the data..
splitting the data
turning data to tensors


## Test Modelling

In [76]:
import torch
import torch.nn as nn

In [77]:
import torch.nn.functional as F

In [78]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [79]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.fc1 = nn.Linear(16 * 25 * 25, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 36)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [80]:
device = torch.device('cuda')

In [81]:
model = Test_Model().to(device)

In [82]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [ ]:
# preds[0]

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [ ]:
EPOCHS = 3

In [ ]:
loss_logs = []

In [ ]:
from tqdm import tqdm

In [ ]:
PROJECT_NAME = "Sign-Language-Recognition"

In [ ]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [ ]:
import wandb

In [ ]:
len(os.listdir('./data/'))

36

In [ ]:
import random

In [ ]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name=NAME)
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()

19


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
df = pd.Series(loss_logs)

In [ ]:
df.plot.line(figsize=(12,6))

In [ ]:
test(model,X_test,y_test)

In [ ]:
test(model,X_train,y_train)

In [ ]:
preds

In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for i in range(len(X_test)):
        real_class = torch.argmax(y_test[i]).to(device)
        net_out = model(X_test[i].view(-1,1,112,112).to(device).float())
        net_out = net_out[0]
        predictied_class = torch.argmax(net_out)
#         print(predictied_class)
        if str(predictied_class) == str(real_class):
            correct += 1
        total += 1
print(round(correct/total,3))

In [ ]:
total

In [ ]:
for real,pred in zip(y_batch,preds):
    print(real)
    print(torch.argmax(pred))
    print('\n')